In [1]:
!pip install --upgrade pip

  Attempting uninstall: pip
    Found existing installation: pip 21.3.1
    Uninstalling pip-21.3.1:
      Successfully uninstalled pip-21.3.1


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Ferhat\\AppData\\Local\\Temp\\pip-uninstall-eywbcl3k\\pip.exe'
Consider using the `--user` option or check the permissions.



In [2]:
!pip install numpy
!pip install pandas
!pip install sklearn
!pip install torch
!pip install transformers

  Using cached sklearn-0.0-py2.py3-none-any.whl


In [4]:
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import transformers
import warnings
warnings.filterwarnings('ignore')
from transformers import AutoTokenizer, AutoModel 
from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertModel, TrainingArguments, Trainer
from torch.utils.data import Dataset, DataLoader

In [7]:
print(np.__version__)
print(pd.__version__)
print(skl.__version__)
print(torch.__version__)
print(transformers.__version__)

1.20.3
1.3.4
0.24.2
1.10.0
4.12.5


In [ ]:
df = pd.read_csv('cleantweets.csv')
df = df[['clean_tweet','NumericalLabel']]
#batch_1 = df[:10000] 
df['NumericalLabel'].value_counts()
labels = df['NumericalLabel']

In [ ]:
print(np.__version__)
print(pd.__version__)
print(skl.__version__)
print(t.__version__)
print(ppb.__version__)

1.18.5
1.1.5
0.24.2
1.6.0
3.5.1


In [ ]:
df.head()

,clean_tweet,NumericalLabel
0,Paul Krugman Nobel Luddite I had to tweak th...,1
1,But dum b a ss said you know nothing abou...,1
2,Another use case for blockchain and Tipper T...,2
3,free coins,2
4,WE are happy to announce that PayVX Presale ...,2


In [ ]:
labels

# GPU 
Declare a variable which will hold the device we’re training on (CPU or GPU)

In [ ]:
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

True


# Preparing the Dataset and Dataloader
We will start with defining few key variables that will be used later during the training/fine tuning stage. Followed by creation of Dataset class - This defines how the text is pre-processed before sending it to the neural network. We will also define the Dataloader that will feed the data in batches to the neural network for suitable training and processing. Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the docs at PyTorch

## Triage Dataset Class
This class is defined to accept the Dataframe as input and generate tokenized output that is used by the DistilBERT model for training.
We are using the DistilBERT tokenizer to tokenize the data in the TITLE column of the dataframe.
The tokenizer uses the encode_plus method to perform tokenization and generate the necessary outputs, namely: ids, attention_mask
To read further into the tokenizer, refer to this document
target is the encoded category on the news headline.
The Triage class is used to create 2 datasets, for training and for validation.
Training Dataset is used to fine tune the model: 80% of the original data
Validation Dataset is used to evaluate the performance of the model. The model has not seen this data during training.
## Dataloader
Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
This control is achieved using the parameters such as batch_size and max_len.
Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 2
LEARNING_RATE = 1e-05
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

In [ ]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        tweet = str(self.data.clean_tweet[index])
        tweet = " ".join(tweet.split())
        inputs = self.tokenizer.encode_plus(
            tweet,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.NumericalLabel[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)
all_set = Triage(df, tokenizer, MAX_LEN)


FULL Dataset: (50859, 2)
TRAIN Dataset: (40687, 2)
TEST Dataset: (10172, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
full_loader = DataLoader(all_set, **train_params)

# Creating the Neural Network for Fine Tuning
## Neural Network
We will be creating a neural network with the DistillBERTClass.
This network will have the DistilBERT Language model followed by a dropout and finally a Linear layer to obtain the final outputs.
The data will be fed to the DistilBERT Language model as defined in the dataset.
Final layer outputs is what will be compared to the encoded category to determine the accuracy of models prediction.
We will initiate an instance of the network called model. This instance will be used for training and then to save the final trained model for future inference.
## Loss Function and Optimizer
Loss Function and Optimizer and defined in the next cell.
The Loss Function is used the calculate the difference in the output created by the model and the actual output.
Optimizer is used to update the weights of the neural network to improve its performance.

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        #self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 4)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = DistillBERTClass()
model.to(device)

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

# Fine Tuning the Model
Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network.

Following events happen in this function to fine tune the neural network:

- The dataloader passes data to the model based on the batch size.
- Subsequent output from the model and the actual category are compared to calculate the loss.
- Loss value is used to optimize the weights of the neurons in the network.
- After every 5000 steps the loss value is printed in the console.

In [ ]:
# Function to calcuate the accuracy of the model

def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print('   ')
    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

Training Loss per 5000 steps: 1.4684494733810425
Training Accuracy per 5000 steps: 25.0
Training Loss per 5000 steps: 0.2781393148569836
Training Accuracy per 5000 steps: 90.7868426314737
Training Loss per 5000 steps: 0.19769605163033585
Training Accuracy per 5000 steps: 93.73812618738125
   
The Total Accuracy for Epoch 0: 93.79900213827513
Training Loss Epoch: 0.19591861839782815
Training Accuracy Epoch: 93.79900213827513
Training Loss per 5000 steps: 0.013458801433444023
Training Accuracy per 5000 steps: 100.0
Training Loss per 5000 steps: 0.054581340301231684
Training Accuracy per 5000 steps: 98.49030193961208
Training Loss per 5000 steps: 0.05049490184169496
Training Accuracy per 5000 steps: 98.64513548645135
   
The Total Accuracy for Epoch 1: 98.64575908766928
Training Loss Epoch: 0.05026594309643077
Training Accuracy Epoch: 98.64575908766928


In [ ]:
#def compute_metrics(pred):
#    labels = pred.label_ids
#    preds = pred.predictions.argmax(-1)
#    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
#    acc = accuracy_score(labels, preds)
#    return {
#        'accuracy': acc,
#        'f1': f1,
#        'precision': precision,
#        'recall': recall
#    }

def compute_metrics(preds, labels):
    #labels = pred.label_ids
    #preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    print(f"Accuracy: {acc}")
    print(f"f1: {f1}")
    print(f"precision: {precision}")
    print(f"recall: {recall}")
    conf_mat = confusion_matrix(labels, preds)
    print(f"Confusion matrix: {conf_mat}")
    cl_report = classification_report(labels, preds)
    print(f"Classification report: {cl_report}")

def valid(model, testing_loader):
    model.eval()
    list = []
    preds = np.array(list)
    labels = np.array(list)
    n_correct = 0; n_wrong = 0; total = 0; tr_loss = 0; nb_tr_steps= 0; nb_tr_examples=0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)
            preds = np.append(preds,big_idx.to("cpu").numpy())
            labels = np.append(labels,targets.to("cpu").numpy())

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 5000 steps: {loss_step}")
                print(f"Validation Accuracy per 5000 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    compute_metrics(preds, labels)
    
    return epoch_accu

In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

Validation Loss per 5000 steps: 0.045048024505376816
Validation Accuracy per 5000 steps: 100.0
Validation Loss per 5000 steps: 0.06187871896327051
Validation Accuracy per 5000 steps: 98.49030193961208
Validation Loss Epoch: 0.061835541611588975
Validation Accuracy Epoch: 98.48604011010617
Accuracy: 0.9848604011010618
f1: [0.96181896 0.98767665 0.98800175]
precision: [0.95730145 0.98578199 0.99102845]
recall: [0.96637931 0.98957861 0.98499348]
Confusion matrix: [[1121   22   17]
 [  22 4368   24]
 [  28   41 4529]]
Classification report:               precision    recall  f1-score   support

         0.0       0.96      0.97      0.96      1160
         1.0       0.99      0.99      0.99      4414
         2.0       0.99      0.98      0.99      4598

    accuracy                           0.98     10172
   macro avg       0.98      0.98      0.98     10172
weighted avg       0.98      0.98      0.98     10172

Accuracy on test data = 98.49%


In [ ]:
# Saving the files for re-use

#output_model_file = './SavedModels/pytorch_distilbert_news.bin'
#output_vocab_file = './SavedModels/vocab_distilbert_news.bin'

#model_to_save = model
#torch.save(model_to_save, output_model_file)
#tokenizer.save_vocabulary(output_vocab_file)

model_dir = './SavedModels/'

# Specify a path
PATH = model_dir + "pytorch_distilbert_sent_uncased.pt"

# Save
torch.save(model, PATH)

#model.save_pretrained(model_dir+'pytorch_distilbert_sent_uncased/model')
tokenizer.save_pretrained(model_dir)

print('All files saved')

All files saved


In [ ]:
model_dir = './SavedModels/'
PATH = model_dir + "pytorch_distilbert_sent_uncased.pt"
tokenizer = DistilBertTokenizer.from_pretrained(model_dir)

In [ ]:
#model = torch.load("./SavedModels/pytorch_distilbert_news.bin")
#tokenizer = DistilBertTokenizer.from_pretrained("./SavedModels/")

# retreive the saved model 
model_dir = './SavedModels/'
PATH = model_dir + "pytorch_distilbert_sent_uncased.pt"
#model = DistilBertForTokenClassification.from_pretrained(model_dir + 'pytorch_distilbert_sent_uncased/model', 
#                                                        local_files_only=True)
model = DistillBERTClass()
model = torch.load(PATH)
model.to(device)
model.eval()

DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_feat

In [ ]:
def Predict(new_tweet, label, tokenizer, model):
    data = [[new_tweet, label]]
 
    # Create the pandas DataFrame
    df = pd.DataFrame(data, columns = ['clean_tweet', 'NumericalLabel']) 
    df_set = Triage(df, tokenizer, MAX_LEN)

    test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

    df_loader = DataLoader(df_set, **test_params)
    model.eval()
    with torch.no_grad():
        for _,data in enumerate(df_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask)
            big_val, big_idx = torch.max(outputs.data, dim=1)
    return big_idx


In [ ]:
new_tweet = "I am neutral to cryptocurrencies"
outputs = Predict(new_tweet, 1, tokenizer, model)
outputs

tensor([1], device='cuda:0')

# Evaluating Model #2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [ ]:
confusion_matrix(test_labels, prediction)

array([[201,  47,  51],
       [ 33, 848, 144],
       [ 23, 165, 988]])

In [ ]:
classification_report(test_labels, prediction)

'              precision    recall  f1-score   support\n\n           0       0.78      0.67      0.72       299\n           1       0.80      0.83      0.81      1025\n           2       0.84      0.84      0.84      1176\n\n    accuracy                           0.81      2500\n   macro avg       0.81      0.78      0.79      2500\nweighted avg       0.81      0.81      0.81      2500\n'

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.460 (+/- 0.00)
